In [1]:
# Includes
from supernovacontroller.sequential import SupernovaDevice
from BinhoSupernova.commands.definitions import *

## Getting Started

#### 1. Create an instance of the Supernova class
To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [2]:
supernova = SupernovaDevice()

#### 2. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(usb_address)
```

- ``usb_address: str (optional)``: The OS HID path assigned to the device. This can be obtained using the ``supernovacontroller.getConnectedSupernovaDevicesList()`` method. The ``usb_address`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``usb_address`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [3]:
info = supernova.open()
print(info)

{'hw_version': 'C', 'fw_version': '2.5.0-58-7f09388', 'serial_number': '99DEAFA5B575B453A98225EE6456C318', 'manufacturer': 'Binho LLC', 'product_name': 'Binho Supernova'}


## I3C Protocol API


### 1. Configuring the Supernova/I2C FRAM Setup

To facilitate the I2C over I3C file transfer test:

**Necessary Firmware:**
- Supernova firmware version 2.4.0
- Supernova SDK version 2.2.0
- Supernova Controller version 1.4.0

**Connection Arrangement:**

<img src="assets/I2C over I3C connection.png" alt="connection_diagram" width="40%">

**Testing Procedure:**

The objective of this test is to load a text file, save it in an I2C FRAM memory, read the I2C FRAM, and store the data into a new text file that can be used to compare contents. I3C with legacy I2C device compatibility will be used to communicate with the I2C FRAM.

To load the file and write the data to the I2C FRAM, the flowchart diagram is the following:

<img src="assets/Write Flowchart.png" alt="write_diagram" width="30%">

To retrieve the data from the I2C FRAM, the flowchart diagram is the following:

<img src="assets/Read Flowchart.png" alt="read_diagram" width="30%">


### 1. Load the text file to transfer

The I2C FRAM supports storing up to 32KB of data. Because of this limitation, the created text file has a size of approximately 30KB and contains the text "Binho Supernova Demo" repeated over and over.

For the data to be transferred, the code must transformed into bytes, the ASCII representation of the letters.


In [4]:
# Read the file contents
with open("Binho_Supernova_Demo.txt", "r") as file:
    file_content = file.read()

# Convert the content into bytes
file_bytes = file_content.encode('utf-8')

### 2. Create interface to manage the Supernova I3C peripheral as controller 


To use any protocol with the Supernova Controller package, the user must create a new interface for the desired protocol. In this case, the demo calls for I3C, and since the Supernova will be the controller of the bus, the "i3c.controller" interface must be used.

In [5]:
i3c = supernova.create_interface("i3c.controller")

### 3. Setting up I3C bus parameters and initializing the I3C bus

Since the Supernova will be communicating with an I2C target, the open drain frequency must be supported by I2C devices, in this case, 1MHz. The push-pull frequency can be set but will not be used for any of the transactions to the I2C device.


In [6]:
i3c.set_parameters(I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ, I2cTransferRate._1MHz)
(success, _) = i3c.init_bus(voltage = 3300)

### 4. Send the data via I3C

To write the data via I3C, the text file is divided into 64-byte sections. Since the I2C FRAM target requires a memory subaddress for the data to be written to, an auxiliary method is used to convert the starting position of each section into a subaddress. This subaddress is then used to correctly place each 64-byte section in the I2C FRAM memory.

#### Define the value to the 2-byte address auxiliary function

In [7]:
def number_to_bytes(num):
    # Ensure the number fits within 2 bytes
    if num < 0 or num > 0xFFFF:
        raise ValueError("Number out of range for 2 bytes")

    # Convert the number to a 2-byte array
    byte1 = (num >> 8) & 0xFF  # MSB
    byte2 = num & 0xFF         # LSB

    return [byte1, byte2]

#### Write the file contents via I3C to the I2C FRAM target

In [8]:
# Initialize variables
package_size = 64
file_length = len(file_bytes)
total_packages = (file_length + package_size - 1) // package_size  # Total packages needed

I2C_FRAM_ADDRESS = 0x50

print("Start the file transfer")
# Load and send 64-byte arrays
for i in range(total_packages):
    start = i * package_size
    end = start + package_size

    # Convert the starting position to a 2-byte array
    subaddress_array = number_to_bytes(start) 
    
    # Section the text file into a package_size section list
    bytes_to_send = list(file_bytes[start:end])

    # Send the bytes_to_send via I3C 
    (success, _) = i3c.write(target_address = I2C_FRAM_ADDRESS, mode = TransferMode.I2C_MODE, subaddress = subaddress_array, buffer= bytes_to_send)
    
    # Handle errors while writing the 
    if not success:
        # Handle the write failure (e.g., retry or abort)
        print("I2C write failed!")
        break

print("Finished the file transfer")

Start the file transfer
Finished the file transfer


### 5. Retrieve the FRAM data and store a new text file

To read the data via I3C, the memory is accessed in 250-byte sections. The process calculates the remaining bytes to be read and adjusts the section size accordingly. Each section is then read from the I2C FRAM, and the retrieved data is appended to a bytearray. This continues until all the data is successfully read, after which the complete data is saved to a text file in ASCII format.

#### Move the I2C FRAM memory pointer to address [0x00, 0x00], which is the starting address of the previously stored text file data

In [9]:
(success, _) = i3c.write(target_address = I2C_FRAM_ADDRESS, mode = TransferMode.I2C_MODE, subaddress = [0x00, 0x00], buffer= [])

#### Read 30KB worth of data from the I2C FRAM

In [10]:
# Initialize variables
total_bytes_to_read = file_length
read_size = 250
bytes_read = 0
read_data = bytearray()  # To store the data read from the I2C FRAM

print("Start the FRAM read")

# Loop to read data in 250-byte sections
while bytes_read < total_bytes_to_read:
    # Calculate the remaining bytes
    remaining_bytes = total_bytes_to_read - bytes_read

    # Calculate the amount of bytes to read
    bytes_to_read = min(read_size, remaining_bytes)  
    
    # Read the I2C FRAM memory
    success, data = i3c.read(target_address = I2C_FRAM_ADDRESS,  mode = TransferMode.I2C_MODE, length = bytes_to_read , subaddress = [])
    
    if success:
        # Append the data to the complete read_data bytearray
        read_data.extend(data)
        
        # Update the count of bytes read
        bytes_read += len(data)
    else:
        # Handle the read failure (e.g., retry or abort)
        print("I2C read failed!")
        break

print("Finished the FRAM read")

Start the FRAM read
Finished the FRAM read


#### Store the read data in the "I2C_Read_Binho_Supernova_Demo.txt" file

In [11]:
# Write the read data to a new text file in ASCII format
output_file = "I2C_Read_Binho_Supernova_Demo.txt"
with open(output_file, "w") as file:
    # Convert the bytearray to a string and write to the file
    file.write(read_data.decode('utf-8'))

#### 7. Compare the original text file with the retrieved one from the I2C FRAM.

To do this, access any text file comparator, upload the original, read the text files, and search for differences.

For testing purposes this can be done in VSCode executing the following cell to compare both files:

In [12]:
import os

# Paths to the files you want to compare
file1 = './Binho_Supernova_Demo.txt'
file2 = './I2C_Read_Binho_Supernova_Demo.txt'

# Command to open VSCode with the diff view
command = f'code --diff {file1} {file2}'

# Execute the command
os.system(command)

0